# Qwen3 1.7B Training Notebook (GPTQ + LoRA Fine-tuning)

This notebook fine-tunes Qwen3-1.7B (GPTQ model) on training data only using LoRA and saves the model for later use.

**Key Changes from BitsAndBytes Version:**
- Uses GPTQ quantized Qwen3-1.7B model (Int4/Int8)
- Uses auto-gptq for quantization handling
- Uses standard LoRA (no DoRA support with GPTQ)
- Trains only on training data (no test-time training)
- Saves the fine-tuned model for later loading

**Benefits of GPTQ + LoRA:**
- **Pre-quantized**: Model already quantized, no dynamic quantization
- **Stable**: GPTQ provides consistent quantization
- **Memory Efficient**: Int4/Int8 quantization reduces VRAM usage
- `r=16` - Rank from TT-1 config
- `lora_alpha=32` - Alpha from TT-1 config
- `lora_dropout=0.1` - Dropout from TT-1 config

In [ ]:
# Install dependencies - GPTQ + LoRA setup
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'trl==0.21.0' 'optimum==1.27.0' 'auto-gptq==0.7.1' 'deepspeed==0.17.4' 'logits-processor-zoo==0.2.1' 'vllm==0.10.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'triton==3.2.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'clean-text'
# Install latest PEFT for LoRA support (no BitsAndBytes needed for GPTQ)
!uv pip install --system --no-index -U --no-deps --find-links='/kaggle/input/jigsaw-packages2/whls/' 'peft' 'accelerate' 'datasets'

print("✅ Dependencies installed for GPTQ + LoRA setup")
print("📁 Model will be loaded from GPTQ dataset on Kaggle")

# 1. Test Drive Training (Verification on Training Data)

This section performs a quick test drive on the first 100 training examples to verify the setup works correctly. The fine-tuned model from this test is **not used** - we reload the original model for the actual training.

In [ ]:
%%writefile constants.py
# GPTQ model path for training
BASE_MODEL_PATH = "/kaggle/input/qwen3-gptq/transformers/1.7b-gptq-int4/1"  # TODO: Update this path
print("✅ Using GPTQ model from Kaggle input")

LORA_PATH = "qwen3_1.7b_gptq_lora_output/"  # GPTQ LoRA output path
FINAL_MODEL_PATH = "qwen3_1.7b_gptq_finetuned/"  # Path for merged final model
DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules/"

POSITIVE_ANSWER = "Yes"
NEGATIVE_ANSWER = "No"
COMPLETE_PHRASE = "Answer:"
BASE_PROMPT = '''You are a moderator of subreddit.  given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.'''

In [ ]:
%%writefile utils.py
import pandas as pd
from datasets import Dataset
from constants import POSITIVE_ANSWER, NEGATIVE_ANSWER, COMPLETE_PHRASE, BASE_PROMPT
import random, numpy as np
random.seed(42)
np.random.seed(42)


def build_prompt(row):
    return f"""
{BASE_PROMPT}

Subreddit: r/{row["subreddit"]}
Rule: {row["rule"]}
Examples:
1) {row["positive_example"]}
{COMPLETE_PHRASE} Yes

2) {row["negative_example"]}
{COMPLETE_PHRASE} No

---
Comment: {row["body"]}
{COMPLETE_PHRASE}"""


def get_training_dataframe(data_path, sample_size=None):
    """Process training data for test drive"""
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    
    if sample_size:
        train_dataset = train_dataset.head(sample_size)
    
    # Process training data
    train_df = train_dataset[["body", "rule", "subreddit", "rule_violation",
                              "positive_example_1","positive_example_2",
                              "negative_example_1","negative_example_2"]].copy()

    # Randomly select positive_example and negative_example
    train_df["positive_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["positive_example_1"],
        train_df["positive_example_2"]
    )
    train_df["negative_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["negative_example_1"],
        train_df["negative_example_2"]
    )

    # Drop original candidate columns
    train_df.drop(columns=["positive_example_1","positive_example_2",
                           "negative_example_1","negative_example_2"], inplace=True)

    return train_df


def get_dataframe_to_train(data_path, training_only=True):
    """Modified: Only use training data when training_only=True"""
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    
    flatten = []

    # ---------- Process training data ----------
    train_df = train_dataset[["body", "rule", "subreddit", "rule_violation",
                              "positive_example_1","positive_example_2",
                              "negative_example_1","negative_example_2"]].copy()

    # Randomly select positive_example and negative_example
    train_df["positive_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["positive_example_1"],
        train_df["positive_example_2"]
    )
    train_df["negative_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["negative_example_1"],
        train_df["negative_example_2"]
    )

    # Drop original candidate columns
    train_df.drop(columns=["positive_example_1","positive_example_2",
                           "negative_example_1","negative_example_2"], inplace=True)

    flatten.append(train_df)
    
    # Changed: Skip test data processing when training_only=True
    if not training_only:
        test_dataset = pd.read_csv(f"{data_path}/test.csv").sample(frac=0.5, random_state=42).reset_index(drop=True)
        
        # ---------- Process test data ----------
        for violation_type in ["positive", "negative"]:
            for i in range(1, 3):
                sub_dataset = test_dataset[["rule","subreddit",
                                            "positive_example_1","positive_example_2",
                                            "negative_example_1","negative_example_2"]].copy()

                if violation_type == "positive":
                    body_col = f"positive_example_{i}"
                    other_positive_col = f"positive_example_{3-i}"
                    sub_dataset["body"] = sub_dataset[body_col]
                    sub_dataset["positive_example"] = sub_dataset[other_positive_col]
                    sub_dataset["negative_example"] = np.where(
                        np.random.rand(len(sub_dataset)) < 0.5,
                        sub_dataset["negative_example_1"],
                        sub_dataset["negative_example_2"]
                    )
                    sub_dataset["rule_violation"] = 1

                else:  # violation_type == "negative"
                    body_col = f"negative_example_{i}"
                    other_negative_col = f"negative_example_{3-i}"
                    sub_dataset["body"] = sub_dataset[body_col]
                    sub_dataset["negative_example"] = sub_dataset[other_negative_col]
                    sub_dataset["positive_example"] = np.where(
                        np.random.rand(len(sub_dataset)) < 0.5,
                        sub_dataset["positive_example_1"],
                        sub_dataset["positive_example_2"]
                    )
                    sub_dataset["rule_violation"] = 0

                sub_dataset.drop(columns=["positive_example_1","positive_example_2",
                                          "negative_example_1","negative_example_2"], inplace=True)

                flatten.append(sub_dataset)

    # Merge all DataFrames
    dataframe = pd.concat(flatten, axis=0)
    dataframe = dataframe.drop_duplicates(ignore_index=True)

    return dataframe


def build_dataset(dataframe):
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)

    columns = ["prompt"]
    if "rule_violation" in dataframe:
        dataframe["completion"] = dataframe["rule_violation"].map(
            {
                1: POSITIVE_ANSWER,
                0: NEGATIVE_ANSWER,
            }
        )
        columns.append("completion")

    dataframe = dataframe[columns]
    dataset = Dataset.from_pandas(dataframe)
    dataset.to_pandas().to_csv("/kaggle/working/training_dataset.csv", index=False)
    return dataset

In [ ]:
%%writefile test_drive.py
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from transformers.utils import is_torch_bf16_gpu_available
from utils import build_dataset, get_training_dataframe
from constants import DATA_PATH, BASE_MODEL_PATH


def test_drive_training():
    print("🚀 Starting test drive training on first 100 training examples...")
    
    # Load first 100 training examples
    dataframe = get_training_dataframe(DATA_PATH, sample_size=100)
    train_dataset = build_dataset(dataframe)
    
    print(f"Test drive dataset size: {len(train_dataset)} samples")
    
    # LoRA configuration for test drive (same as TT-1)
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
        # No use_dora=True - GPTQ doesn't support DoRA
    )
    print("✅ LoRA config created for test drive")
    
    # Load the GPTQ model (no quantization config needed - already quantized)
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        local_files_only=True,
    )
    print("✅ GPTQ model loaded for test drive")
    
    # Short training config for test drive
    training_args = SFTConfig(
        num_train_epochs=1,  # Very short training
        
        per_device_train_batch_size=2,  # Smaller for GPTQ
        gradient_accumulation_steps=8,
        
        optim="paged_adamw_8bit",
        learning_rate=1e-4,
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        
        bf16=is_torch_bf16_gpu_available(),
        fp16=not is_torch_bf16_gpu_available(),
        dataloader_pin_memory=True,
        
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
    
        save_strategy="no",  # Don't save test drive model
        output_dir="test_drive_output/",
        logging_steps=10,
        report_to="none",
    
        completion_only_loss=True,
        packing=False,
        remove_unused_columns=False,
    )
    print("✅ Test drive training config created")
    
    # Create trainer for test drive
    trainer = SFTTrainer(
        model=base_model,
        args=training_args,
        train_dataset=train_dataset,
        peft_config=lora_config,
    )
    
    # Run test drive training
    trainer.train()
    print("✅ Test drive training completed successfully!")
    print("📝 Setup verified - proceeding to main training...")
    
    # Clean up to free memory
    del trainer, base_model
    torch.cuda.empty_cache()
    
    return True


if __name__ == "__main__":
    test_drive_training()

In [ ]:
# Run test drive training
!python test_drive.py

# 2. Main Training with GPTQ + LoRA

Now that the setup is verified, we proceed with the actual training using the GPTQ model.

In [ ]:
%%writefile train.py
import pandas as pd
import torch

from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm
from transformers.utils import is_torch_bf16_gpu_available
from utils import build_dataset, get_dataframe_to_train
from constants import DATA_PATH, BASE_MODEL_PATH, LORA_PATH, FINAL_MODEL_PATH


def main():
    print("📝 Note: Using original GPTQ model (not the test drive fine-tuned model)")
    
    # Changed: Only use training data (training_only=True)
    dataframe = get_dataframe_to_train(DATA_PATH, training_only=True)
    train_dataset = build_dataset(dataframe)
    
    print(f"Training dataset size: {len(train_dataset)} samples")
    
    # LoRA configuration with settings from TT-1
    lora_config = LoraConfig(
        r=16,  # From TT-1 config
        lora_alpha=32,  # From TT-1 config
        lora_dropout=0.1,  # From TT-1 config
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
        # No use_dora=True - GPTQ doesn't support DoRA
    )
    print("✅ LoRA config created with TT-1 settings")
    
    # Load the GPTQ model (no quantization config - already pre-quantized)
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        local_files_only=True,
    )
    print("✅ GPTQ model loaded from local Kaggle input")
    
    # Optimized training config for GPTQ + LoRA
    training_args = SFTConfig(
        num_train_epochs=2,  # Keep same epochs
        
        # GPTQ batch sizes (typically lower than 4-bit due to higher memory usage)
        per_device_train_batch_size=2,  # GPTQ uses more VRAM
        gradient_accumulation_steps=8,  # Higher accumulation to compensate
        
        optim="paged_adamw_8bit",  # Keep 8-bit optimizer for memory efficiency
        learning_rate=1e-4,  # Same learning rate as TT-1
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        
        bf16=is_torch_bf16_gpu_available(),
        fp16=not is_torch_bf16_gpu_available(),
        dataloader_pin_memory=True,
        
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
    
        save_strategy="epoch",
        save_steps=500,
        output_dir=LORA_PATH,
        logging_steps=50,
        report_to="none",
    
        completion_only_loss=True,
        packing=False,
        remove_unused_columns=False,
    )
    print("✅ Training config created for GPTQ + LoRA")
    
    # Create SFTTrainer with the loaded GPTQ model
    trainer = SFTTrainer(
        model=base_model,
        args=training_args,
        train_dataset=train_dataset,
        peft_config=lora_config,
    )
    
    print("🚀 Starting GPTQ + LoRA training...")
    trainer.train()
    
    # Save the LoRA adapters
    trainer.save_model(LORA_PATH)
    print(f"✅ LoRA adapters saved to: {LORA_PATH}")
    
    # Merge and save the final model for easier loading
    print("🔄 Merging LoRA adapters with GPTQ base model...")
    
    # Load base GPTQ model for merging
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        local_files_only=True,
    )
    print("✅ GPTQ base model loaded for merging")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, local_files_only=True)
    
    # Load and merge LoRA adapters
    peft_model = PeftModel.from_pretrained(base_model, LORA_PATH)
    merged_model = peft_model.merge_and_unload()
    
    # Save merged model
    merged_model.save_pretrained(FINAL_MODEL_PATH)
    tokenizer.save_pretrained(FINAL_MODEL_PATH)
    
    print(f"✅ Final GPTQ + LoRA merged model saved to: {FINAL_MODEL_PATH}")
    print("🎉 GPTQ + LoRA training completed successfully!")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile accelerate_config.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 8  # Higher accumulation for GPTQ
  gradient_clipping: 1.0
  train_batch_size: 32  # Same effective batch size: 2*8*2 = 32
  train_micro_batch_size_per_gpu: 2  # Lower for GPTQ memory usage
  
  zero_stage: 2
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  
  stage3_gather_16bit_weights_on_model_save: false
  stage3_max_live_parameters: 1e8
  stage3_max_reuse_distance: 1e8
  stage3_prefetch_bucket_size: 5e7
  stage3_param_persistence_threshold: 1e5
  
  zero_allow_untested_optimizer: true
  zero_force_ds_cpu_optimizer: false
  
  fp16:
    enabled: true
    loss_scale: 0
    initial_scale_power: 16
    loss_scale_window: 1000
    hysteresis: 2
    min_loss_scale: 1
  
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
  dynamo_use_fullgraph: false
  dynamo_use_dynamic: false
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

In [ ]:
!accelerate launch --config_file accelerate_config.yaml train.py

In [ ]:
# Check training output and model files
import os
print("GPTQ LoRA adapter files:")
!ls -la qwen3_1.7b_gptq_lora_output/
print("\nFinal GPTQ merged model files:")
!ls -la qwen3_1.7b_gptq_finetuned/

In [ ]:
# Create a compressed archive for easier upload to Kaggle datasets
!tar -czf qwen3_1.7b_gptq_lora_model.tar.gz qwen3_1.7b_gptq_finetuned/
print("GPTQ LoRA model archived as: qwen3_1.7b_gptq_lora_model.tar.gz")
print("Upload this file to Kaggle as a dataset for use in the inference notebook.")

# ⚡ Speed Optimization Guide for 2x T4 GPUs (28GB Total VRAM) - GPTQ Edition

## Current Settings Status: ✅ **GOOD** for 2x T4 GPUs with GPTQ
- **Memory**: GPTQ Int4 model + LoRA fits in ~10-14GB per GPU
- **Batch Size**: 2 per device × 8 accumulation = 32 effective batch size
- **DeepSpeed**: ZeRO Stage 2 with FP16 - optimal for this setup
- **Speed**: Stable with GPTQ pre-quantization

## 🚀 Additional Speed Optimizations for GPTQ:

### **Quick Wins:**
1. **Increase Batch Size Carefully** (Cell 9 - `train.py`):
   ```python
   per_device_train_batch_size=3,  # Can try 3-4 if VRAM allows
   gradient_accumulation_steps=6,   # Adjust accordingly
   ```
   
2. **Faster Optimizer** (Cell 9 - `train.py`):
   ```python
   optim="adamw_torch_fused",  # If PyTorch 2.0+
   ```

3. **Reduce LoRA Rank** (Cell 9 - `train.py`):
   ```python
   r=8,              # Can reduce from 16 to 8
   lora_alpha=16,     # Adjust proportionally
   ```

## 💡 **GPTQ vs BitsAndBytes Comparison:**
1. **GPTQ**: Pre-quantized, stable, no DoRA support
2. **Memory**: GPTQ uses more VRAM than 4-bit BitsAndBytes
3. **Speed**: GPTQ is stable but potentially slower than dynamic quantization
4. **Compatibility**: Works with standard LoRA, established workflow